In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

from helper_functions import elementlines, nearestlines

In [ ]:
hs.load

In [ ]:
%matplotlib qt

### Variables that you need to change

- path
    - path to the data
- file
    - file name
- elements
    - list of elements to be analyzed
- lines_of_interest
    - lines of interest, will be included in the output
- info_on_all_lines
    - boolean, if true, the output will contain all lines, not only the lines of interest
- zero_peak
    - boolean, if true, the zero peak will be sliced out
- zero_peak_end_index
    - index of the last element of the zero peak, which will be sliced out
- line_ratio_list
    - list of list of lines which will be given a ratio, eg. K to L
    - [['Ni_Ka', 'Ni_La'], ['Ni_Ka', 'Ni_Kb']]
- fwtm_to_fwhm_lines
    - lines of interest for the fwtm to fwhm ratio, should not be overlapping with other peaks
    - the fwhm and fwtm here is taken from the raw spectrum, since the fitted gaussians have perfect shapes
    - then if a overlapping peak is used, the ratio will be skewed

In [ ]:
# TODO: detector setup (I_beam, distance, etc.)

In [ ]:
# #### SU9000 #####
path = 'data/Skomedal_2022-03-23_EDS_SU9000'
file = 'Spectrum 03.emsa'  # 30 kV
elements = ['Al', 'Au', 'C', 'Cu', 'F', 'Fe', 'Mg', 'Mo', 'Ni', 'O', 'Si', 'Sn']  # from Mari, pluss Mg and Sn
lines_of_interest = ['Al_Ka', 'C_Ka', 'Cu_Ka', 'Mo_La', 'Ni_Ka', 'Ni_La', 'O_Ka', 'Si_Ka']
info_on_all_lines = False
line_ratio_list = [['Ni_Ka', 'Ni_La'], ['Ni_Ka', 'Mo_Ka'], ['Mo_Ka', 'Mo_La']]
fwtm_to_fwhm_lines = ['Ni_Ka']

# file = 'Spectrum 09.emsa'  # 15 kV

# file = 'Spectrum 10.emsa'  # 10 kV
# file = 'Spectrum 11.emsa'  # 5 kV


##### GaAs 30 kV APREO #####
# path = 'data/Maehlum_2022-09-06_EDS-SEM-APREO'
# file = 'GaAs_30kV.emsa'
# elements = ['Ga', 'As', 'O', 'C', 'Si']
# lines_of_interest = ['Ga_Ka', 'As_Ka', 'O_Ka', 'C_Ka', 'Si_Ka', 'Ga_La', 'As_La', 'Ga_Kb', 'As_Kb', 'Ga_Ll']
# info_on_all_lines = False
# line_ratio_list = [['As_Ka', 'As_La'], ['Ga_Ka', 'Ga_La']]
# fwtm_to_fwhm_lines = ['Ga_Ka', 'Ga_La']

# # file = 'GaAs_15kV.emsa'
# # file = 'GaAs_10kV.emsa'


# ##### Mo APREO #####
# path = 'data/Mæhlum_2022-09-06_EDS-SEM-APREO'
# file = 'Mo_30kV.emsa'
# elements = ['Mo', 'O']
# lines_of_interest = ['Mo_Ka', 'O_Ka', 'Mo_La', 'Mo_Kb', 'Mo_Ll', 'Mo_Lb1', 'Mo_Lg3']
# info_on_all_lines = False
# line_ratio_list = [['Mo_Ka', 'Mo_La']]
# fwtm_to_fwhm_lines = ['Mo_Ka', 'Mo_La']

zero_peak = True 
zero_peak_end_index = 33

In [ ]:
# function arguments that you can change, eg. if a step take too long

# the model
model_background_order = 6

# calibration
# change xray_lines from "all_alpha" to certain lines, or "all"
# change rounds to do more or less rounds of calibration


In [ ]:
s.plot(True)

## The output parameters that will be calculated are:


In [ ]:
# output parameters
scale = np.nan  # float
offset = np.nan  # float 
energy_resolution = np.nan  # float
total_counts = np.nan  # float
background_counts = np.nan  # float
dh_limit = np.nan # float
lines_info = dict() # dict of dicts as {line_name1: {true_energy: ..., calib_energy: ..., area: ..., max_counts: ..., sigma: ..., fwhm: ...}, line_name2: {...}}
line_ratios = dict()  # dict as {line1a/line1b: ratio1, line2a/line2b: ratio2, ...}
fwtm_to_fwhm = dict()  # dict as {line1: fwtm_to_fwhm1, line2: fwtm_to_fwhm2, ...}


## Loading the data and setting some parameters

In [ ]:
s = hs.load(path + '/' + file, signal_type='EDS_TEM')  # have to pretent it's a TEM signal
s.add_elements(elements)
if zero_peak:
    s = s.isig[zero_peak_end_index:]

Vacc = s.metadata.Acquisition_instrument.TEM.beam_energy
x_max = s.axes_manager[0].high_value  # highest x-axis value in keV, used in Duane-Hunt
x = s.axes_manager[0].axis  # x-axis in keV

In [ ]:
s.metadata

In [ ]:
# TODO: add these as user inputs and add them to the output dict
# also metadata like Vacc etc.

date = s.metadata.General.date
time = s.metadata.General.time
original_filename = s.metadata.General.original_filename
name = f'{file[:-5]}'  # remove ".emsa", TODO: fix a nicer name

In [ ]:
plt.rcParams['font.size'] = '16'
plt.rcParams["figure.figsize"] = (15,8)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.family'] = 'monospace'

In [ ]:
# plot the spectrum
# s.plot(xray_lines=True)

In [ ]:
# these are temporary arrays used to show the effect of the calibrations
# temporary because thay are not used in the final output

scale_list = [s.axes_manager[0].scale]
offset_list = [s.axes_manager[0].offset]
energy_res_list = [s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa]

In [ ]:
# # Duane-Hunt method to find the real E_0
def calculate_duane_hunt(buffer_start=2, buffer_end=0.1, xaxis_plot_buffer=0.5, dh_plot=False):
    if Vacc > x_max:
        print(f'Vacc={Vacc} > x_max={x_max}, Duane-Hunt not possible')
        return np.nan
    else:
        s_dh = s.deepcopy() 
        # making the lin fit of the background right before Vacc
        dh_start = Vacc-buffer_start
        dh_end = Vacc-buffer_end
        s_end = s_dh.isig[dh_start:dh_end] # slice with keV
        m_end = s_end.create_model(auto_background=False)
        m_end.add_polynomial_background(order=1)
        m_end.fit()

        x_s_end = s_dh.isig[dh_start-xaxis_plot_buffer:dh_end+xaxis_plot_buffer].axes_manager[0].axis
        dh_bg_zero_index = np.argmin(np.abs(m_end[-1].function(x_s_end) * s_dh.axes_manager[0].scale))

        dh_limit = x_s_end[dh_bg_zero_index]
        print(f'Duane-Hunt limit: {dh_limit:.3f} keV')

        if dh_plot:
            plt.plot(x, s_dh.data, label='spectrum', marker='o')
            plt.plot(x_s_end, m_end[-1].function(x_s_end)* s_dh.axes_manager[0].scale, label='bg lin. fit')
            plt.plot(s_end.axes_manager[0].axis, s_end.data, marker='o', label=f'points in bg lin. fit')
            plt.axhline(0, color='k', linestyle='--')
            plt.axvline(dh_limit, color='r', linestyle='--', label=f'Duane-Hunt: {dh_limit:.3f} keV')
            plt.axvline(Vacc, color='y', linestyle='--', label=f'Vacc from instrument: {Vacc} keV', alpha=0.6)
            plot_buffer = 1.1
            plt.title(f'{file}\nlin fit range kV: [{dh_start:.2f}, {dh_end:.2f}]')
            plt.ylim(-5, m_end[-1].function(x_s_end).max() * s_dh.axes_manager[0].scale*plot_buffer)
            plt.xlim(Vacc-buffer_start*plot_buffer, dh_limit + buffer_end)
            plt.legend()
            plt.show()

        return dh_limit


In [ ]:
dh_limit = calculate_duane_hunt(dh_plot=False)

In [ ]:
# Using Duane-Hunt to slice the spectrum
def use_dh_to_slice_spectrum(dh_limit=dh_limit, s=s, plot=False):
    if np.isnan(dh_limit):
        print('No Duane-Hunt limit found, not slicing the spectrum')
    else:
        s = s.isig[:dh_limit]
        print(f'Spectrum sliced at {dh_limit:.2f} keV')
        if plot:
            s.plot(xray_lines=True)
    return s

In [ ]:
s = use_dh_to_slice_spectrum(dh_limit, plot=False)
x = s.axes_manager[0].axis  # x-axis in keV, after slicing

In [ ]:
# creating a model
m = s.create_model(auto_background=False)

# fitting the data
m.add_polynomial_background(order=model_background_order) 
m.fit_background()
m.fit(bounded=True)
'm'
# m.plot(True)

In [ ]:
m.plot(True)

## Calibration (scale, offset and resolution)

In [ ]:
#  Functions for calibrating the energy axis

def calibrate_axis(rounds=2, xray_lines='all_alpha', scale=scale, offset=offset):
    # calibration of the energy axis, i.e. scale and offset
    print('Calibrating energy axis (with many elements it can take multiple minutes)')
           
    for i in range(rounds):
        print(f'Calibrating offset and scale, round {i+1} of {rounds}')
        m.calibrate_energy_axis(calibrate='offset', xray_lines=xray_lines)
        offset_list.append(s.axes_manager[0].offset)
        m.calibrate_energy_axis(calibrate='scale', xray_lines=xray_lines)
        scale_list.append(s.axes_manager[0].scale)

    print(f'Scale: {scale_list[-1]:.6f} eV/px \nOffset: {offset_list[-1]:.6f} keV')

    return scale_list[-1], offset_list[-1]


def calibrate_resolution(rounds=1, xray_lines='all_alpha'):
    # calibration of the energy resolution
    for i in range(rounds):
        print(f'Calibrating energy resolution, round {i+1} of {rounds}')
        m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
        energy_res_list.append(s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa)

    print(f'Calibrated energy resolution: {energy_res_list[-1]:.3f} eV')
    return energy_res_list[-1]


def print_calibration_info():
    # make pretty print of calibration info
    infos = [' ', 'Scale [eV/channel]', 'Offset [keV]', 'E-res [eV]']
    row1 = ['Current', f'{scale_list[-1]:.6f}', f'{offset_list[-1]:.6f}', f'{energy_res_list[-1]:.3f}']
    row2 = ['Original', f'{scale_list[0]:.6f}', f'{offset_list[0]:.6f}', f'{energy_res_list[0]:.3f}']
    row3 = ['Δ original', f'{(scale_list[-1] - scale_list[0])/scale_list[-2]*100:.3f} %', 
    f'{(offset_list[-1] - offset_list[0])/offset_list[-2]*100:.3f} %', f'{(energy_res_list[-1] - energy_res_list[0])/energy_res_list[0]*100:.3f} %']
    row4 = ['Δ last step', f'{(scale_list[-1] - scale_list[-2])/scale_list[-2]*100:.3f} %', 
    f'{(offset_list[-1] - offset_list[-2])/offset_list[-2]*100:.3f} %', f'{(energy_res_list[-1] - energy_res_list[-2])/energy_res_list[-2]*100:.3f} %']

    for i in range(len(infos)):
        print(f'{infos[i]:<20}{row1[i]:<15}{row2[i]:<15}{row3[i]:<15}{row4[i]:<15}')
    


In [ ]:
[compo.name for compo in m.xray_lines]

In [ ]:
# test of 'all_alpha' in m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
m.calibrate_energy_axis(calibrate='resolution', xray_lines='all_alpha')
s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
# test of 'all_alpha' in m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['Ni_La'])
s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
# test of 'all_alpha' in m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['O_Ka'])
s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
# test of 'all_alpha' in m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['Cu_Ka'])
s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
# test of 'all_alpha' in m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['Al_Ka'])
s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
# test of 'all_alpha' in m.calibrate_energy_axis(calibrate='resolution', xray_lines=xray_lines)
m.calibrate_energy_axis(calibrate='resolution', xray_lines=['Mo_Ka'])
s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa

In [ ]:
s.plot(True)

In [ ]:
s.metadata

In [ ]:
s.plot(True)

In [ ]:
m.active_components[1].A.value

In [ ]:
s.plot(True)

In [ ]:
s.metadata

In [ ]:
scale, offset = calibrate_axis()
energy_resolution = calibrate_resolution()

In [ ]:
print_calibration_info()

## Peak positions fitting

In [ ]:
# helper functions


def energy_index(energy):
    # given an energy in keV, return the index of that energy in the spectrum
    return np.abs(x - energy).argmin()


def energy_counts(energy):
    # given an energy in keV, return the counts at that energy
    energy_index = energy_index(energy)
    return s.data[energy_index]


def range_counts(start, stop):
    # given a range in keV, return the counts in that range
    return s.data[energy_index(start), energy_index(stop)].sum()


def max_counts(energy, buffer=10):
    # get the highest counts in +- 10 channels around a given energy
    i = np.argmin(np.abs(x - energy))
    return s.data[i-buffer:i+buffer].max()
     

def gaussian_counts(line):
    # give the sum of counts from a gaussian fit of a line
    return (m[line].function(x) * s.axes_manager[0].scale).sum()

def gaussian_max_counts(line):
    # give the max counts from a gaussian fit of a line
    return m[line].function(x).max() * s.axes_manager[0].scale


def bg_point(energy):
    # give the background counts at a certain energy
    return m[-1].function(energy) * s.axes_manager[0].scale


def bg_range(range_array):
    # give the background sum counts in range, where range is an array of energies
    return (m[-1].function(range_array) * s.axes_manager[0].scale).sum()

    
def kev_to_channels(energy):
    return np.argmin(np.abs(x - energy))


def theoretical_energy(line):
    # returns the theoretical energy of a given line, e.g. 'Ga_Ka'
    element = line.split('_')[0]
    line_name = line.split('_')[1]
    return hs.material.elements[element]['Atomic_properties']['Xray_lines'][line_name]['energy (keV)']


In [ ]:
# functions for calibrations of lines

def calibrate_lines(rounds=2, xray_lines='all_alpha'):
    # calibrating the lines using HyperSpy
    # calibrate energy rounds times, and width and sub_weight only once
    for i in range(rounds):
        print(f'Calibrating peak positions, round {i+1} of {rounds}')
        m.calibrate_xray_lines(calibrate='energy', xray_lines=xray_lines, kind='single') # use kind='multi' for better results?
        if i == 0:
            m.calibrate_xray_lines(calibrate='width', xray_lines=xray_lines, kind='single')
            m.calibrate_xray_lines(calibrate='sub_weight', xray_lines=xray_lines, kind='single')


def make_lines_info(all_lines=info_on_all_lines):
    # make dict with lines info
    lines_info = {}
    for i in range(len(m) - 1): # last component is the background
        if (all_lines == True) or (m[i].name in lines_of_interest):
            lines_info[m[i].name] = {
                'true_energy': theoretical_energy(m[i].name),
                'calib_energy': m[i].centre.value,
                'area': gaussian_counts(i),
                'max_counts': gaussian_max_counts(i),
                'HS_height': m[i].height * scale,
                'sigma': m[i].sigma.value,
                'fwhm': m[i].fwhm * 1000,
                'Fiori P/B': m[i].A.value / (m[-1].function(m[i].centre.value) * scale)
                
            }
    return lines_info


def print_lines_info(give_return=False):
    # print line info
    headers = ['Line', 'True E [keV]', 'Calib. E [keV]', 'Area [counts]', 'Max (fit)', 'Max (HS)', 'Sigma [keV]', 'FWHM [eV]', 'Fiori P/B']
    table = ''
    for i in range(len(headers)):
        table += f'{headers[i]:<15}'
    table += '\n'
    for line in lines_info:
    #   # this looks bad, but it is to align the columns and adjust the number of decimals
        one_line_info = f'{line:<15}'
        one_line_info += f'{lines_info[line]["true_energy"]:<15.4f}'
        one_line_info += f'{lines_info[line]["calib_energy"]:<15.4f}'
        one_line_info += f'{lines_info[line]["area"]:<15.1f}'
        one_line_info += f'{lines_info[line]["max_counts"]:<15.1f}'
        one_line_info += f'{lines_info[line]["HS_height"]:<15.1f}'
        one_line_info += f'{lines_info[line]["sigma"]:<15.4f}'
        one_line_info += f'{lines_info[line]["fwhm"]:<15.4f}'
        one_line_info += f'{lines_info[line]["Fiori P/B"]:<15.1f}'
        table += one_line_info + '\n'
    
    if give_return == True:
        return table
    else:
        print(table)

In [ ]:
# m.components.Ni_Ka.fwhm

In [ ]:
# NiKa = m.components.Ni_Ka.A.value

# NiKaBG = m.as_signal(component_list=["background_order_6"])
# NiKaRange = (7.16, 7.76) 
# cf = (NiKaRange[1]-NiKaRange[0])/scale 
# roi = hs.roi.SpanROI(7.16, 7.76)
# B = roi(NiKaBG)
# PB = NiKa/B.integrate1D(-1).data[0]
# PB_F = cf * PB 

# print("The peak to background ratio is: ", PB_F)

In [ ]:
# (m[-1].function(s.isig[7.16:7.76].axes_manager[0].axis) * s.axes_manager[0].scale).sum()

In [ ]:
# PB

In [ ]:
# B.integrate1D(-1).data[0]

In [ ]:
# type(roi)

In [ ]:
# cf

In [ ]:
calibrate_lines(rounds=2, xray_lines='all_alpha')

In [ ]:
lines_info = make_lines_info()
print_lines_info()

In [ ]:
m['Ni_Ka'].A.value

In [ ]:
m['Ni_Ka'].A.value / (m[-1].function(m['Ni_Ka'].centre.value)*scale)

In [ ]:
m[-1].function(m['Ni_Ka'].centre.value) * scale

In [ ]:
m['Ni_Ka'].height * s.axes_manager[0].scale

In [ ]:
## line ratios

def peak_ratio(line1, line2):
    # give the K to L ratio of a line, e.g. 'Ga_Ka' to 'Ga_La'
    try:
        m[line1]
        m[line2]
    except ValueError:
        print('line not in model:', line1, line2)
        return np.nan
    return gaussian_counts(line1) / gaussian_counts(line2)

def calculate_all_line_ratios():
    # calculate the K to L ratios for line_ratio_list
    line_ratios = {}
    for line_pair in line_ratio_list:
        pair_name = line_pair[0] + '/' + line_pair[1]
        line_ratios[pair_name] =  peak_ratio(line_pair[0], line_pair[1])
    return line_ratios

In [ ]:
line_ratios = calculate_all_line_ratios()
line_ratios

In [ ]:
## FWTM to FWHM

def fwtm_to_fwhm_line(line, plot_ratio=False):
    try:
        m[line]
    except ValueError:
        print('line not in model:', line)
        return {'fwhm': np.nan, 'fwtm': np.nan, 'fwtm/fwhm': np.nan}
    s_wo_bg = s.deepcopy()
    s_wo_bg.data = s.data - m[-1].function(x)*s.axes_manager[0].scale
    window = m[line].sigma.value * 4 # window around the line
    line_energy = m[line].centre.value
    s_line_window = s_wo_bg.isig[line_energy-window:line_energy+window]
    fwhm = s_line_window.estimate_peak_width(factor=0.5, parallel=False).data[0]
    fwtm = s_line_window.estimate_peak_width(factor=0.1, parallel=False).data[0]

    # TODO: Put this plotting in a separate function?
    if plot_ratio:
        plt.plot(s_line_window.axes_manager[0].axis, s_line_window.data, label=f'Signal without bg', marker='o')
        plt.axvline(line_energy-window, color='k', linestyle='--', label='window')
        plt.axvline(line_energy+window, color='k', linestyle='--')
        plt.axvline(theoretical_energy(line), color='r', linestyle='--', label='theoretical line center')
        plt.axvline(m[line].centre.value, color='g', linestyle='--', label='calibrated center')
        plt.plot([line_energy-fwhm/2, line_energy+fwhm/2], [lines_info[line]['max_counts']/2, lines_info[line]['max_counts']/2], label='fwhm')
        plt.plot([line_energy-fwtm/2, line_energy+fwtm/2], [lines_info[line]['max_counts']/10, lines_info[line]['max_counts']/10], label='fwtm')
        plt.title(f'{line} \n FWHM: {fwhm*1000:.2f} eV, FWTM: {fwtm*1000:.2f} eV, ratio: {fwtm/fwhm:.3f}')
        plt.legend()
        plt.show()
    
    return {'fwhm': fwhm*1000, 'fwtm': fwtm*1000, 'fwtm/fwhm': fwtm/fwhm}


def fwtm_to_fwhm_all():
    fwtm_to_fwhm = {}
    for line in fwtm_to_fwhm_lines:
        fwtm_to_fwhm[line] = fwtm_to_fwhm_line(line, plot_ratio=False)

    return fwtm_to_fwhm
        

fwtm_to_fwhm = fwtm_to_fwhm_all()
fwtm_to_fwhm

## Total counts and background counts

In [ ]:
total_counts = s.data.sum()
# total_counts_m = m.signal.data.sum() # does not work, m.signal is just a view of s

# counts in the background
background_counts = bg_range(x).sum()

# counts in the spectrum without background
total_counts_wo_bg = total_counts - background_counts


def print_count_info():
    print(f'background_counts: {background_counts:.1f}')
    print(f'total_counts: {total_counts:.1f}')
    print(f'total_counts_wo_bg: {total_counts_wo_bg:.1f}')

    # ratio of total counts to total counts in background
    print(f'total_counts/background_counts: {total_counts/background_counts:.1f}')

    # ratio of total counts to total counts in spectrum without background
    print(f'total_counts/total_counts_wo_bg: {total_counts/total_counts_wo_bg:.1f}')

    # percentage of counts in peaks compared to total counts
    print(f'percentage of counts in peaks: {total_counts_wo_bg/total_counts*100:.1f}%')


In [ ]:
def print_output(scale=scale, offset=offset, energy_resolution=energy_resolution, total_counts=total_counts, 
background_counts=background_counts, dh_limit=dh_limit, lines_info=lines_info, line_ratios=line_ratios, fwtm_to_fwhm=fwtm_to_fwhm):
    print(f"{'file:':<20} {file}")
    print(f"{'scale:':<20} {scale:.4f}")
    print(f"{'offset:':<20} {offset:.4f}")
    print(f"{'energy_resolution:':<20} {energy_resolution:.4f}")
    print(f"{'total_counts:':<20} {total_counts:.1f}")
    print(f"{'background_counts:':<20} {background_counts:.1f}")
    print(f"{'dh_limit:':<20} {dh_limit:.3f}")
    print(print_lines_info(give_return=True))
    print(f"line_ratios: {line_ratios}")
    print(f"fwtm_to_fwhm: {fwtm_to_fwhm}")
    print_count_info()


In [ ]:
print_output()


In [ ]:
# put output in a dictionary

def output_dict(scale=scale, offset=offset, energy_resolution=energy_resolution, total_counts=total_counts,
background_counts=background_counts, lines_info=lines_info, line_ratios=line_ratios, fwtm_to_fwhm=fwtm_to_fwhm, 
data=date, time=time, original_filename=original_filename, name=name, elements=elements):
    output = {'name': name, 'date': date, 'time': time, 'original_filename': original_filename,'scale': scale, 
    'elements': elements, 'energy_resolution': energy_resolution, 'offset': offset,  'total_counts': total_counts,
    'background_counts': background_counts, 'dh_limit':dh_limit, 'lines_info': lines_info, 'line_ratios': line_ratios, 
    'fwtm_to_fwhm': fwtm_to_fwhm}
    return output


In [ ]:
output = output_dict()
output

In [ ]:
# saving the output to a csv

import csv

with open(f'results/{name}_{date}.csv', 'w') as f:
    for key in output.keys():
        f.write("%s,%s\n"%(key,output[key]))

In [ ]:
m.family_lines

In [ ]:
plt.close('all')

In [ ]:
# plotting all components and their centers, labeled

plt.plot(s.axes_manager[0].axis, s.data-m[-1].function(s.axes_manager[0].axis)*scale+100, label='spectrum', marker='o')
# m_data = np.zeros_like(s.data)

for comp in m:
    if comp == m[-1]:
        continue
    # m_data += comp.function(s.axes_manager[0].axis)*scale

    # # comment in to only plot centain lines
    # if not 'Ni_' in comp.name:
    #     continue
    
    line_y = comp.height * scale
    line_x = comp.centre.value
    true_e  = hs.material.elements[comp.name.split('_')[0]].Atomic_properties.Xray_lines.as_dictionary()[comp.name.split('_')[1]]['energy (keV)']

    plt.plot([line_x, line_x], [0, line_y+100])
    plt.plot([true_e, true_e], [0, line_y+100], '--', color='k', alpha=0.7)

    # plt.plot(s.axes_manager[0].axis, comp.function(s.axes_manager[0].axis)*scale+m[-1].function(s.axes_manager[0].axis)*scale, label=comp.name)
    plt.plot(s.axes_manager[0].axis, comp.function(s.axes_manager[0].axis)*scale+100, label=comp.name)

    plt.text(line_x, line_y+100, f'{comp.name}, {comp.centre.value:.3f}',rotation=90)

# plt.plot(s.axes_manager[0].axis, m_data+150, label='model')


plt.ylim(15,25e4)
plt.yscale('log')

# plt.legend()
plt.show()

In [ ]:
s.plot(True)